In [1]:
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt
import time
import random
from TTHelperFuncts import *

In [2]:
# computeSiteAssignmentLSALP
   # this linear Program takes the cluster bounds and the matrix of distances that were already computed and generates
   # a linear program to minimize the the distance sums taking the cluster bounds into account
def computeSiteAssignmentLSALP(upperBound, lowerBound, distances):
    m = Model("assignment")
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.Params.LogToConsole = 0
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1),lb = 0.0, ub = 1.0, name = "x") #vtype=GRB.BINARY, name="x")
    #objective
    m.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MINIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)    
    m.optimize()
    return m

# computeSiteAssignmentRadialLP
   # this linear Program takes the cluster bounds and the matrix of distances that were already computed and generates
   # a linear program to maximize the the distance sums taking the cluster bounds into account
def computeSiteAssignmentRadialLP(upperBound, lowerBound, distances):
    m = Model("assignmentRad")
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.Params.LogToConsole = 0
    m.Params.IterationLimit = np.inf
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1),lb=0.0, ub=1.0, name="x")
    #objective
    m.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MAXIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)
    m.optimize()
    return m

# computeSiteAssignmentRadialLPSeeded
   # This LP specifies the start vector for the variables to perform a site assignment
def computeSiteAssignmentRadialLPSeeded(upperBound, lowerBound, distances, assignment):
    m = Model("assignment")
    m.Params.LogToConsole = 0
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1), lb=0.0,ub=1.0, name="x")
    #warm start
    for i in range(np.size(distances, axis = 0)):
        for j in range(np.size(distances, axis = 1)):
            x[i,j].start = assignment[np.size(distances,axis = 0)*i + j]
    #objective
    m.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MAXIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)
    
    m.optimize()
    return m

# computeRadialForBounds
   # This function updates the right hand side of the provided gurobi LP model and reoptimized
def computeRadialForBounds(k,n,upperLimBound, lowerLimBound, radDistance, cbasis, vbasis, model):
    model.Params.LogToConsole = 0
    #variables
    rhs = []
    for i in range(k):
        rhs.append(upperLimBound[i])
        rhs.append(lowerLimBound[i])
    for i in np.ones(n).tolist():
        rhs.append(i)
    model.setAttr("RHS",model.getConstrs(),rhs)
    model.update()
    #objective
    model.setAttr("cbasis",model.getConstrs(),cbasis)
    model.setAttr("vbasis",model.getVars(),vbasis)
    model.Params.IterationLimit = np.inf
    model.Params.Method = 0
    model.Params.Presolve = 0
    model.Params.Sifting = 0
    model.update()
    model.optimize()
    return model

*Cluster set up*

In [3]:
# setUpCluster
   # This function stores our collection of example problems that can be specified in the main executable code block
   # if you specify a test case that we do not have an error will be thrown. please set up any test case you wish to
   # to run in this function
def setUpCluster(sites, points, smallest, biggest, fixed=True):
    x = np.random.rand(points)
    y = np.random.rand(points)
    randPoints = []
    sitesStart = []
    sitesEnd = []
    if sites == 5 and fixed:
        xStart = [
            (0.25, 0.75),
            (0.25, 0.25),
            (0.75, 0.25),
            (0.75, 0.75),
            (0.5, 0.5)
            ]
        xEnd = [
            (0.25, 0.55),
            (0.45, 0.25),
            (0.75, 0.45),
            (0.62, 0.62),
            (0.38, 0.63)
        ]
        for i in range(sites):
            #fixed path
            sitesStart.append((xStart[i][0]*float(points) , xStart[i][1]*float(points)))
            sitesEnd.append(((xEnd[i][0])*float(points) , (xEnd[i][1])*float(points)))
    elif sites == 10 and fixed:
        xStart = [
            (0.2, 0.8),
            (0.2, 0.2),
            (0.5, 0.15),
            (0.8, 0.2),
            (0.8, 0.8),
            (0.5, 0.85),
            
            (0.35, 0.35),
            (0.35, 0.65),
            (0.65, 0.65),
            (0.65, 0.35)
            ]
        xEnd = [
            (0.2, 0.6),
            (0.35, 0.16),
            (0.6, 0.17),
            (0.8, 0.4),
            (0.65, 0.83),
            (0.35, 0.84),
            
            (0.35, 0.45),
            (0.55, 0.65),
            (0.65, 0.55),
            (0.55, 0.35)
            ]
        for i in range(sites):
            #fixed path
            sitesStart.append((xStart[i][0]*float(points) , xStart[i][1]*float(points)))
            sitesEnd.append(((xEnd[i][0])*float(points) , (xEnd[i][1])*float(points)))
    elif sites == 20 and fixed:
        xStart = [
            (0.1, 0.125),
            (0.1, 0.375),
            (0.1, 0.625),
            (0.1, 0.875),
            (0.3, 0.125),
            (0.3, 0.375),
            (0.3, 0.625),
            (0.3, 0.875),
            (0.5, 0.125),
            (0.5, 0.375),
            (0.5, 0.625),
            (0.5, 0.875),
            (0.7, 0.125),
            (0.7, 0.375),
            (0.7, 0.625),
            (0.7, 0.875),
            (0.9, 0.125),
            (0.9, 0.375),
            (0.9, 0.625),
            (0.9, 0.875)
            ]
        xEnd = [
            (0.13, 0.1),
            (0.07, 0.41),
            (0.13, 0.641),
            (0.08, 0.855),
            (0.33, 0.1),
            (0.27, 0.41),
            (0.28, 0.641),
            (0.33, 0.855),
            (0.53, 0.1),
            (0.47, 0.41),
            (0.53, 0.641),
            (0.48, 0.855),
            (0.73, 0.1),
            (0.67, 0.41),
            (0.73, 0.641),
            (0.68, 0.855),
            (0.93, 0.1),
            (0.87, 0.41),
            (0.93, 0.641),
            (0.88, 0.855)
            ]
        for i in range(sites):
            #fixed path
            sitesStart.append((xStart[i][0]*float(points) , xStart[i][1]*float(points)))
            sitesEnd.append(((xEnd[i][0])*float(points) , (xEnd[i][1])*float(points)))
    else:
        #full random
        xStart = np.random.rand(sites)
        yStart = np.random.rand(sites)
        xEnd = np.random.rand(sites)
        yEnd = np.random.rand(sites)
        shift = 1.0/(2.0*float(sites))
        for i in range(sites):
            #full random
            sitesStart.append((xStart[i]*float(points) , yStart[i]*float(points)))
            sitesEnd.append(((xStart[i]+((xEnd[i] -0.5)*shift))*float(points) , 
                             (yStart[i]+((yEnd[i]-0.5)*shift))*float(points)))
            
    for i in range(points):
        randPoints.append((x[i]*float(points) , y[i]*float(points)))
        
    startB = np.random.randint(smallest, biggest, size=(sites,))
    while sum(startB) != points: 
        startB = np.random.randint(smallest, biggest, size=(sites,))
    endB = np.random.randint(smallest, biggest, size=(sites,))
    while sum(endB) != points: 
        endB = np.random.randint(smallest, biggest, size=(sites,))
        
    uBound = np.copy(startB)
    lBound = np.copy(startB)
    for i in range(len(startB)):
        if startB[i] > endB[i]:
            uBound[i] = startB[i]
            lBound[i] = endB[i]
        else:
            uBound[i] = endB[i]
            lBound[i] = startB[i]
    return sitesStart, sitesEnd, randPoints, uBound, lBound, startB, endB

# startingClusters
   # This function takes the test case that was specified and generates all the other data that is needed 
   # to begin computing the transition
def startingClusters(sStart, sEnd, points, upper, lower, start, end):
    #starting LSA assignment
    lsaDistance = nonWeightedDistanceLSA(sStart, points)
    lsaAssignment = computeSiteAssignmentLSALP(start, start, lsaDistance)
    x = lsaAssignment.x
    lsaStartVect = x
    lsaStart = genCurPointAssign(x, len(sStart), len(points))
    #starting Radial assignment
    radDistance = nonWeightedDistanceRadial(sStart, points)
    radAssignment = computeSiteAssignmentRadialLP(upper, lower, radDistance)
    x = radAssignment.x
    radStart = genCurPointAssign(x, len(sStart), len(points))
    #ending LSA assignment
    lsaDistance = nonWeightedDistanceLSA(sEnd, points)
    lsaAssignment = computeSiteAssignmentLSALP(end, end, lsaDistance)
    x = lsaAssignment.x
    lsaEndVect = x
    testlsaEnd = np.zeros(len(points))
    lsaEnd = genCurPointAssign(x, len(sStart), len(points))

    #ending Radial assignment
    radDistance = nonWeightedDistanceRadial(sEnd, points)
    radAssignment = computeSiteAssignmentRadialLP(upper, lower, radDistance)
    x = radAssignment.x
    radEnd = genCurPointAssign(x, len(sStart), len(points))
    return lsaStart, radStart, lsaStartVect, lsaEnd, radEnd, lsaEndVect


### Fixed LSA2 Radial ###

In [4]:
# modelSeed
   # This function precomputes the basis vectors we need from gurobi to use warm start methods
def modelSeed(sites, points, upper, lower, lsaBound):
    #starting LSA assignment
    lsaDistance = nonWeightedDistanceLSA(sites, points)
    lsaAssignment = computeSiteAssignmentLSALP(lsaBound, lsaBound, lsaDistance)
    lsaCBasis = lsaAssignment.cbasis
    lsaVBasis = lsaAssignment.vbasis
            
    #starting Radial assignment
    radDistance = nonWeightedDistanceRadial(sites, points)
    radAssignment = computeSiteAssignmentRadialLP(upper, lower, radDistance)
    
    return lsaCBasis, lsaVBasis, radAssignment.cbasis, radAssignment.vbasis

# LSA2RadialNew
   # Algorithm 2 in the paper
def LSA2RadialNew(k,n,sites, points, lsaAssign, upper, lower, model, radAssign, lsaModel,radDistance):
    # begin with current assignment and cluster sizes
    curAssign = np.copy(lsaAssign)
    currentAssignCount = np.zeros(k)
    for i in range(n):
        currentAssignCount[int(curAssign[i])] += 1.
    #test if we are already done
    if sameAssignment(lsaAssign, radAssign):
        return 0,0, True
    
    # counting time
    timeSum = 0
    iterCount = 0
    count = 0
    start = time.time()
    previousObjectiveValue = lsaModel.getObjective().getValue()
    cbasis = lsaModel.cbasis
    vbasis = lsaModel.vbasis
    finalObjectiveValue = model.getObjective().getValue()

    model.reset()
    while previousObjectiveValue < finalObjectiveValue:
        count += 1 
        oldAssignCount = np.copy(currentAssignCount)
        oldAssign = np.copy(curAssign)
        
        changeOptions = getAllPairsOfClusters(k,oldAssignCount,lower,upper)

        alreadytried = []
        while True:
            choice = random.choice(changeOptions)
            
            upperLimBound, lowerLimBound = genLimitedBounds(choice, oldAssignCount)
            model = computeRadialForBounds(k,n,upperLimBound, lowerLimBound, radDistance, cbasis, vbasis, model)
            currentObjectiveValue = model.getObjective().getValue()
            if previousObjectiveValue < currentObjectiveValue:
                x = model.x
                for i in range(k):
                    for j in range(n):
                        if x[n*i + j] == 1.0:
                            curAssign[j] = i
                currentAssignCount = np.zeros(k)
                for i in range(n):
                    currentAssignCount[int(curAssign[i])] += 1
                break      
        previousObjectiveValue = np.copy(currentObjectiveValue)
        cbasis = model.cbasis
        vbasis = model.vbasis
        if count == 1:
            comparisoncounter = 0
            for i in range(len(curAssign)):
                if not oldAssign[i] == curAssign[i]:
                    comparisoncounter += 1
            if comparisoncounter == 1:
                count -=1       
        previousX = np.copy(model.x)
    timeSum += time.time() - start    
    iterCount+= count
    return iterCount, timeSum, True    


### LSA to Radial function ###

In [5]:
def LSA2Radial(k,n,sites, points, upperBounds, lowerBounds, lsaBound, lsaAssign, radAssign):
    sLSAcBasis, sLSAvBasis, sRadcBasis, sRadvBasis = modelSeed(end, points, upperBounds, lowerBounds, lsaBound)

    lsaDistance = nonWeightedDistanceLSA(sites, points)
    lsaModel = computeSiteAssignmentLSALP(lsaBound, lsaBound, lsaDistance)
    radDistance = nonWeightedDistanceRadial(sites, points)
    radModel = computeSiteAssignmentRadialLP(upperBounds, lowerBounds, radDistance)
    iterCount, startTime, usable = LSA2RadialNew(k,n,sites, points, lsaAssign,
                                                upperBounds, lowerBounds, radModel, radAssign, lsaModel,radDistance)
    return iterCount, startTime, time.time(), usable

### Radial to Radial Step ###

In [6]:
# computeLambda
   # This function finds the next lambda value as described in the paper for the next time a shift could occur
def computeLambda(upperBound, lowerBound, k,n, primalObjFun, primalObjFunValue, DeltaObjFun, primalSolution):
    m = Model("breakRad")
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.Params.LogToConsole = 0
    m.Params.IterationLimit = np.inf
    
    #variables
    y1 = m.addVars(n, lb = -GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="y1") # each item assigned
    y2 = m.addVars(k, lb = -GRB.INFINITY, ub=0.0, vtype=GRB.CONTINUOUS, name="y2") #lower bounds
    y3 = m.addVars(k, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="y3") # upper bounds

    breakpoint = m.addVar(lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name = "breakpoint") # the breakpoint
    
    #objective
    m.setObjective(breakpoint, GRB.MAXIMIZE)
    #constraints
    for i in range(k):
        for j in range(n):
            m.addConstr(y1[j]+y2[i]+y3[i] - DeltaObjFun[i,j]*breakpoint >= primalObjFun[i,j])
    m.addConstr(quicksum(y1[i] for i in range(n))+quicksum(y2[i]*lowerBound[i] for i in range(k))+quicksum(y3[i]*upperBound[i] for i in range(k))
                     -breakpoint*quicksum(DeltaObjFun[i,j]*primalSolution[i*n+j] for i in range(k) for j in range(n))
                    ==primalObjFunValue )
    
    m.optimize()
    return m

# computeLambdaReuse
   # Once the model exists it is computationally faster to reuse it. This function modifies the model as  
   # necessary
def computeLambdaReuse(model, k,n, primalObjFun, primalObjFunValue, DeltaObjFun, primalSolution):
    constraints = model.getConstrs()
    breakpoint = model.getVarByName("breakpoint") 
    quickSumTerm = 0.0
    #constraints
    for i in range(k):
        for j in range(n):
            model.chgCoeff(constraints[i*n+j], breakpoint, (-1)*DeltaObjFun[i,j])
            constraints[i*n+j].setAttr("RHS",primalObjFun[i,j])
            quickSumTerm += DeltaObjFun[i,j]*primalSolution[i*n+j]
            #rhs.append(primalObjFun[i,j])
    model.chgCoeff(constraints[k*n], breakpoint, (-1)*quickSumTerm)
    constraints[k*n].setAttr("RHS",primalObjFunValue)
    model.optimize()
    return model

# Radial2Radial
   # Algorithm 3 in the paper
def Radial2Radial(start, end, points, upperBounds, lowerBounds, step):
    startTime = time.time()
    iterCount = 0.0
    timeCount = 0.0
    #compute starting distance and start = current
    radCurrentDistance = nonWeightedDistanceRadial(start, points)
    radCurrentModel = computeSiteAssignmentRadialLP(upperBounds, lowerBounds, radCurrentDistance)

    #compute ending distance and radial clustering
    radEndDistance = nonWeightedDistanceRadial(end, points)
    radEndModel = computeSiteAssignmentRadialLP(upperBounds, lowerBounds, radEndDistance)

    #this stays fixed. current still is the same as start
    DeltaDistance = radEndDistance - radCurrentDistance

    counter = 0
    totallambda = 0

    while(True):
        counter = counter + 1
        continueValue = False
               
        # compute the primal objective function value and get the current solution
        obj = radCurrentModel.getObjective()
        primalObjValue = obj.getValue()
        primalSolutionVector = radCurrentModel.x
    
        # find next lambda. (called newBreak)
        if counter <=1 :
            lambdaCurrentModel=computeLambda(upperBounds, lowerBounds, k,n, radCurrentDistance, primalObjValue, DeltaDistance, primalSolutionVector)
        else:
            lambdaCurrentModel =computeLambdaReuse(lambdaCurrentModel,k,n, radCurrentDistance, primalObjValue, DeltaDistance, primalSolutionVector)
        iterCount += lambdaCurrentModel.IterCount
        timeCount += lambdaCurrentModel.Runtime
        newBreak = lambdaCurrentModel.getVarByName('breakpoint').X

        # update distances
        newDistance = radCurrentDistance + (step*newBreak)*DeltaDistance
  
        totallambda = totallambda + (1-totallambda)*step*newBreak
    
        radCurrentDistance = newDistance
        DeltaDistance = radEndDistance - radCurrentDistance
    
        # compute new clustering with arm start
        radNextModel = computeSiteAssignmentRadialLPSeeded(upperBounds, lowerBounds, radCurrentDistance,primalSolutionVector)
    
        test = 0
        DiffBetween=np.zeros((k, n))
        for i in range(k):
            for j in range(n):
                DiffBetween[i,j]=radCurrentModel.x[i*n+j]-radNextModel.x[i*n+j]
                if(DiffBetween[i,j] != 0.0):
                    test += 1
        if(test == 0):
            
            break
        if(totallambda > 1):
            break
        radCurrentModel = radNextModel
        curAssign = np.zeros(n)
        x = radNextModel.x
        for i in range(k):
            for j in range(n):
                if x[n*i + j] == 1.0:
                    curAssign[j] = i
        tempSites = getCurrentSites(start, end, totallambda)
    totalTime = time.time() - startTime 
    return counter, totalTime, (iterCount/float(counter)),(timeCount/float(counter))

In [ ]:
# This value can be 5, 10 or 20 to use predetermined sites, any other number will make everything random
k = 10

#this can be any value
n = 100

# these should bound the lower and upper sides of k/n
small = 7
big = 13

#this is the number of runs you want to perform
total = 5

#these are just setting up the run information
epsilon = 1.05
timeSumRR = 0
stepSumRR = 0
timeSumLRStart = 0
stepSumLRStart = 0
timeSumRLEnd = 0
stepSumRLEnd = 0
sliceSteps = 0
diffCount = 0
count = 0
while count < total:
    start, end, points, upperBounds, lowerBounds, startBound, endBound = setUpCluster( k, n, small, big)

    sAssignmentLSA, sAssignmentRad, sAssignmentVect, eAssignmentLSA, eAssignmentRad, eAssignmentVect = startingClusters(start, end, points, upperBounds, 
                                                                                     lowerBounds, startBound, endBound)

    print("the number of points that change are  " + str(diffCounter(sAssignmentLSA, eAssignmentLSA)))
    diffCount += diffCounter(sAssignmentLSA, eAssignmentLSA)
    print("Step1 LSA to Radial Start")
    IterCountStart, LSAStartTime, LSAStartEndTime, usable = LSA2Radial(k,n,start, points, upperBounds, lowerBounds, startBound, sAssignmentLSA, sAssignmentRad)


    timeSumLRStart += LSAStartTime
    stepSumLRStart += IterCountStart
    print("Step2 LSA to Radial End")
    IterCountEnd, LSAEndTime, LSAEndEndTime, usable = LSA2Radial(k,n,end, points, upperBounds, lowerBounds, endBound, eAssignmentLSA, eAssignmentRad)
    print("Step3 Radial to Radial")
    stepCountRadial, totalTimeRRStep, avgRRIters, avgRRRunTime = Radial2Radial(start, end, points, upperBounds,
                                 lowerBounds,epsilon)
    timeSumRR += totalTimeRRStep
    stepSumRR += stepCountRadial
    timeSumLRStart += LSAStartTime
    stepSumLRStart += IterCountStart
    timeSumRLEnd += LSAEndTime
    stepSumRLEnd += IterCountEnd
    count +=1


print("average time to compute LRStart")
print(timeSumLRStart/float(count))
print("average steps computing LRStart")
print(stepSumLRStart/float(count))
print("average time to compute LREnd")
print(timeSumRLEnd/float(count))
print("average steps computing LREnd")
print(stepSumRLEnd/float(count))
print("average RR run time")
print(timeSumRR/float(count))
print("average steps computing RR breakpoints")
print(stepSumRR/float(count))

Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
the number of points that change are  58
Step1 LSA to Radial Start
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Preso